In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from matplotlib import rc
import math
import ast
import re
import seaborn as sns
from IPython.display import Image
%matplotlib inline
plt.rcParams
plt.rcParams['figure.figsize'] = (5,5)
plt.rcParams['font.size'] = 8
plt.rcParams['figure.dpi'] = 100
import warnings;
warnings.filterwarnings('ignore');

In [2]:
import re
import itertools
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans,DBSCAN,SpectralClustering
from textblob import TextBlob
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import VotingClassifier,RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC
from sklearn.semi_supervised import LabelPropagation
from sklearn.multiclass import OneVsRestClassifier,OneVsOneClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,make_scorer,silhouette_score,silhouette_samples
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler,ClusterCentroids,CondensedNearestNeighbour,NeighbourhoodCleaningRule,NearMiss
from collections import Counter
import joblib
import datetime as dt
from datetime import datetime
from scipy.spatial.distance import cosine
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [15]:
from surprise import NormalPredictor
from surprise import Reader
from surprise.model_selection import cross_validate,KFold
from surprise import KNNBasic,SVD, NMF
from surprise import accuracy
from surprise import Dataset, evaluate

In [4]:
df = pd.read_excel('../data/processed/data_model.xlsx')

In [9]:
df.head()

,INSURED_ID,BUYER_ID,PERSON_ID,ENTRY_AGE,COVER_AGE,PAY_AGE,OWNER_AGE,PAY_TERM,COVER_TERM,PRODUCT_CODE,...,POL_ST_lapsed - apl,POL_ST_lapsed - eti,POL_ST_lapsed - rpu,POL_ST_matured,POL_ST_not taken up,POL_ST_postponed,POL_ST_surrendered,POL_ST_surrendered - ecv,POL_ST_surrendered - rpu,POL_ST_with uw requirements
0,PART_ID_10751,0,PART_ID_10751,34,35,35,34,1,1,2161,...,0,0,0,0,0,0,1,0,0,0
1,PART_ID_8104,0,PART_ID_8104,53,58,58,53,5,5,2198,...,0,0,0,0,0,0,0,0,0,0
2,PART_ID_15437,0,PART_ID_15437,59,60,60,59,1,1,2163,...,0,0,0,0,0,0,0,0,0,0
3,PART_ID_5553,PART_ID_578,PART_ID_578,19,24,24,50,5,5,2127,...,0,0,0,0,0,0,0,0,0,0
4,PART_ID_5553,PART_ID_578,PART_ID_578,19,24,24,50,5,5,1476,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df_purchase_history_raw = df.groupby('INSURED_ID').agg({'PRODUCT_CODE': lambda code: list(code),
                                'PRODUCT_CATEGORY': lambda product:  "%s" % ','.join(product)})

In [291]:
df_purchase_history_raw.head(10)

,PRODUCT_CODE,PRODUCT_CATEGORY
INSURED_ID,,
PART_ID_10,[2174],Retirement
PART_ID_10000,[2163],Investment
PART_ID_10001,[2163],Investment
PART_ID_10002,"[2413, 1543]","Retirement,Protection"
PART_ID_10003,"[2413, 1476, 1537]","Retirement,Protection,Protection"
PART_ID_10004,"[2304, 2313, 2332, 2319]","Retirement,Protection,Protection,Protection"
PART_ID_10005,"[2413, 1476, 1543]","Retirement,Protection,Protection"
PART_ID_10006,[2413],Retirement
PART_ID_10007,[1460],Retirement


In [106]:
purchase_category = [i.split(',') for i in  df_purchase_history_raw['PRODUCT_CATEGORY']]

In [123]:
te = TransactionEncoder()
te_ary = te.fit(purchase_category).transform(purchase_category)
df_purchase_category  = pd.DataFrame(te_ary, columns=te.columns_)

In [124]:
df_purchase_category.head()

,Health,Investment,Protection,Retirement,Savings
0,False,False,False,True,False
1,False,True,False,False,False
2,False,True,False,False,False
3,False,False,True,True,False
4,False,False,True,True,False


In [125]:
frequent_itemsets = apriori(df_purchase_category, min_support=0.0001, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.005603,(Health),1
1,0.518922,(Investment),1
2,0.450869,(Protection),1
3,0.332792,(Retirement),1
4,0.082508,(Savings),1
5,0.000244,"(Health, Investment)",2
6,0.005522,"(Protection, Health)",2
7,0.002680,"(Retirement, Health)",2
8,0.002030,"(Health, Savings)",2
9,0.076011,"(Protection, Investment)",2


In [73]:
frequent_itemsets[frequent_itemsets['length']>1]

,support,itemsets,length
5,0.000244,"(Health, Investment)",2
6,0.005522,"(Protection, Health)",2
7,0.002680,"(Retirement, Health)",2
8,0.002030,"(Health, Savings)",2
9,0.076011,"(Protection, Investment)",2
10,0.007065,"(Retirement, Investment)",2
11,0.003005,"(Investment, Savings)",2
12,0.224622,"(Retirement, Protection)",2
13,0.063343,"(Protection, Savings)",2
14,0.041335,"(Retirement, Savings)",2


In [359]:
output_apriori = pd.DataFrame(frequent_itemsets[frequent_itemsets['length']>1])
output_apriori.to_excel('../data/processed/output/apriori.xlsx')

In [224]:
df_purchase_category.columns

Index(['INSURED_ID', 'Health', 'Investment', 'Protection', 'Retirement',
       'Savings'],
      dtype='object')

In [6]:
df_purchase_history_compound = df_purchase_history_raw
cols = ['Health', 'Investment', 'Protection', 'Retirement','Savings']
for c in cols:
    df_purchase_history_compound[c] = df_purchase_history_compound['PRODUCT_CATEGORY'].apply(lambda x: x.split(',').count(c))

In [7]:
df_purchase_history_compound.head()

,PRODUCT_CODE,PRODUCT_CATEGORY,Health,Investment,Protection,Retirement,Savings
INSURED_ID,,,,,,,
PART_ID_10,[2174],Retirement,0,0,0,1,0
PART_ID_10000,[2163],Investment,0,1,0,0,0
PART_ID_10001,[2163],Investment,0,1,0,0,0
PART_ID_10002,"[2413, 1543]","Retirement,Protection",0,0,1,1,0
PART_ID_10003,"[2413, 1476, 1537]","Retirement,Protection,Protection",0,0,2,1,0


In [9]:
df_purchase_compound = df_purchase_history_compound[['Health', 'Investment', 'Protection', 'Retirement','Savings']]

In [297]:
purchase_similar = pd.DataFrame(index=df_purchase_compound.columns,columns=df_purchase_compound.columns)

In [298]:
for i in range(0,len(purchase_similar.columns)) :
    for j in range(0,len(purchase_similar.columns)) :
          purchase_similar.ix[i,j] = 1-cosine(df_purchase_compound.ix[:,i],df_purchase_compound.ix[:,j])

In [299]:
purchase_similar

,Health,Investment,Protection,Retirement,Savings
Health,1,0.00281114,0.156664,0.041304,0.182044
Investment,0.00281114,1,0.104131,0.017485,0.0153525
Protection,0.156664,0.104131,1,0.540787,0.363784
Retirement,0.041304,0.017485,0.540787,1,0.241497
Savings,0.182044,0.0153525,0.363784,0.241497,1


In [300]:
purchase_chain = pd.DataFrame(index=purchase_similar.columns,columns=range(1,6))
 
# Loop through our similarity dataframe and fill in neighbouring item names
for i in range(0,len(purchase_similar.columns)):
    purchase_chain.ix[i,:5] = purchase_similar.ix[0:,i].sort_values(ascending=False)[:5].index

In [301]:
purchase_chain 

,1,2,3,4,5
Health,Health,Savings,Protection,Retirement,Investment
Investment,Investment,Protection,Retirement,Savings,Health
Protection,Protection,Retirement,Savings,Health,Investment
Retirement,Retirement,Protection,Savings,Health,Investment
Savings,Savings,Protection,Retirement,Health,Investment


In [405]:
purchase_sims = np.zeros(df_purchase_compound.shape)

In [406]:
for i in range(len(purchase_sims)):
    for j in df_purchase_compound.columns:
        col_index = (list(df_purchase_compound.columns)).index(j)
        if df_purchase_compound.iloc[i][j] > 0:
            purchase_sims[i][col_index] = 0
        else:
            product_top_names = [str(k) for k in  purchase_chain.ix[j][1:].values]
            product_top_sims = purchase_similar.ix[j].sort_values(ascending=False)[1:].values
            user_purchases = df_purchase_compound.iloc[i][product_top_names].values
            purchase_sims[i][col_index] = sum(user_purchases*product_top_sims)/sum(product_top_sims)

In [407]:
purchase_sims

array([[0.10789331, 0.12508984, 0.46404919, 0.        , 0.30086451],
       [0.00734319, 0.        , 0.08935467, 0.02078891, 0.01912663],
       [0.00734319, 0.        , 0.08935467, 0.02078891, 0.01912663],
       ...,
       [0.00734319, 0.        , 0.08935467, 0.02078891, 0.01912663],
       [0.92635867, 1.61501989, 0.        , 0.        , 1.2072906 ],
       [0.51712599, 0.87005487, 0.        , 0.        , 0.75407756]])

In [432]:
userid_purchases = pd.DataFrame(purchase_sims,columns = df_purchase_compound.columns,index=df_purchase_compound.index)
userid_purchases.head()

,Health,Investment,Protection,Retirement,Savings
INSURED_ID,,,,,
PART_ID_10,0.107893,0.125090,0.464049,0.000000,0.300865
PART_ID_10000,0.007343,0.000000,0.089355,0.020789,0.019127
PART_ID_10001,0.007343,0.000000,0.089355,0.020789,0.019127
PART_ID_10002,0.517126,0.870055,0.000000,0.000000,0.754078
PART_ID_10003,0.926359,1.615020,0.000000,0.000000,1.207291


In [461]:
rec_output = []
for i in range(len(userid_purchases)):
    tmp = [userid_purchases.iloc[i].name]
    d = {k:v for k,v in userid_purchases.iloc[i].to_dict().items() if v!=0}
    tmp.extend(sorted(d, key=d.get, reverse=True))
    rec_output.append(tmp)

In [467]:
output_cosine = pd.DataFrame(rec_output,columns=['INSURED_ID','FIRST CHOICE','SECOND CHOICE','THIRD CHOICE','FOURTH CHOICE'])

In [19]:
output_cosine.head()

,INSURED_ID,FIRST CHOICE,SECOND CHOICE,THIRD CHOICE,FOURTH CHOICE
0,PART_ID_10,Protection,Savings,Investment,Health
1,PART_ID_10000,Protection,Retirement,Savings,Health
2,PART_ID_10001,Protection,Retirement,Savings,Health
3,PART_ID_10002,Investment,Savings,Health,NaN
4,PART_ID_10003,Investment,Savings,Health,NaN


In [469]:
output_cosine.to_excel('../data/processed/output/cosine.xlsx',index=None)

In [10]:
df_surprise = pd.melt(df_purchase_compound.reset_index(),id_vars=['INSURED_ID'], value_vars=['Health', 'Investment', 'Protection', 'Retirement', 'Savings'])

In [11]:
reader = Reader(rating_scale=(1, 11))

In [12]:
data_surprise = Dataset.load_from_df(df_surprise,reader)

In [495]:
cross_validate(NormalPredictor(), data_surprise , cv=5)

{'test_rmse': array([0.9609387 , 0.95775781, 0.9644909 , 0.96815749, 0.96134462]),
 'test_mae': array([0.82379908, 0.82030948, 0.83015826, 0.82625524, 0.82788959]),
 'fit_time': (0.1311025619506836,
  0.1308917999267578,
  0.14737701416015625,
  0.13541340827941895,
  0.13291525840759277),
 'test_time': (0.14199280738830566,
  0.14493799209594727,
  0.15033197402954102,
  0.1506059169769287,
  0.15816998481750488)}

In [13]:
kf = KFold(n_splits=3)
sim_options = {
    'name': 'cosine',
    'user_based': True
}
 
algo = KNNBasic(sim_options=sim_options)
for trainset, testset in kf.split(data_surprise ):
    algo.fit(trainset)
    predictions = algo.test(testset)
    accuracy.rmse(predictions, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9078
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9255
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9121


In [14]:
algo = SVD()
cross_validate(algo, data_surprise, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9173  0.9102  0.9125  0.9158  0.9142  0.9140  0.0025  
MAE (testset)     0.7838  0.7832  0.7797  0.7871  0.7819  0.7831  0.0024  
Fit time          2.47    2.37    2.38    2.38    2.48    2.42    0.05    
Test time         0.06    0.06    0.06    0.06    0.06    0.06    0.00    


{'test_rmse': array([0.91726661, 0.91024387, 0.91245134, 0.91578526, 0.91416549]),
 'test_mae': array([0.78380548, 0.78318637, 0.7797174 , 0.78708708, 0.78191668]),
 'fit_time': (2.4714252948760986,
  2.3716063499450684,
  2.3835785388946533,
  2.381608724594116,
  2.4833571910858154),
 'test_time': (0.05984139442443848,
  0.05987358093261719,
  0.05888676643371582,
  0.05883193016052246,
  0.05983996391296387)}